In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:

#%%
# Funtion lấy một bảng với các thông tin năm, khu vực và chỉ số
# Trường hợp này lấy 1 bảng là thông tin chỉ số PCI trong một năm
def lay_pci_1_nam(url, nam, khu_vuc, chi_so):
    data = pd.read_html(url.format(khu_vuc, chi_so, nam))
    data = data[0]
    return data

#%%
# Funtipn lấy và ghép chỉ số PCI của tất cả các năm
def lay_pci_vn(url, nam, khu_vuc, chi_so):
    pci = pd.DataFrame()
    for nam in nam:
        data = lay_pci_1_nam(url, nam, khu_vuc, chi_so)
        data['nam'] = nam
        pci = pci.append(data)
    return pci

#%%
# Funtion xác định bảng giá trị đầu tiên thu thập của các địa phương
# Trong lollipop plot dự định vẽ 2 điểm cho mỗi địa phương:
# - Điểm đầu tiên bắt đầu đánh giá
# - Điểm của năm hiện tại
# Tuy nhiên các địa phương có thời điểm bắt đầu đánh giá khác nhau
# Vì vậy cần xác định bẳng giá trị đầu tiên đánh giá và năm đầu tiên đánh giá (gọi là bang gia tri khoi dau)
def xac_dinh_khoi_dau(data):
    data_khoi_dau = pd.DataFrame()
    for i in data['dia_phuong'].unique():
        data_dia_phuong = data[data['dia_phuong'] == i]
        nam_dau = data_dia_phuong['nam'].min()
        gia_tri_dau = data_dia_phuong[data_dia_phuong['nam'] == nam_dau]
        data_khoi_dau = data_khoi_dau.append(gia_tri_dau)
    return data_khoi_dau
#%% -----------------
# Xác số hạng thay đổi mỗi năm của mỗi nước
def xac_dinh_thay_doi_thu_hang_1_dia_phuong(data_dia_phuong):
    data = data_dia_phuong.sort_values(by= 'nam', ascending=True).reset_index(drop=True)
    for i in range(data.shape[0]):
        if i == 0:
            data['thay_doi_hang_theo_nam'] = 0
            data['thay_doi_diem_theo_nam'] = 0
        else:
            thay_doi_hang = data['xep_hang'].iloc[i] - data['xep_hang'].iloc[i-1]
            data['thay_doi_hang_theo_nam'].iloc[i] = thay_doi_hang

            thay_doi_diem = data['pci'].iloc[i] - data['pci'].iloc[i-1]
            data['thay_doi_diem_theo_nam'].iloc[i] = thay_doi_diem
    return data
#%%
def bo_sung_thay_doi_theo_nam(data):
    pci_vn = pd.DataFrame()
    dia_phuong = data['dia_phuong'].unique()
    for i in dia_phuong:
        df_dia_phuong = data[data['dia_phuong'] == i]
        df_dia_phuong = xac_dinh_thay_doi_thu_hang_1_dia_phuong(df_dia_phuong)
        pci_vn = pci_vn.append(df_dia_phuong)
    return pci_vn.sort_values(by= ['nam', 'xep_hang'], ascending=True).reset_index(drop=True)

#%%
# Xác định hạng thay đổi cao nhất trong năm
def tim_thay_doi_hang_toi_da_theo_nam(data):
    data = data[['nam', 'thay_doi_hang_theo_nam']]
    data = data.rename( columns = {'thay_doi_hang_theo_nam': 'thay_doi_toi_da'})
    data['thay_doi_toi_da'] = data['thay_doi_toi_da'].abs()
    data = data.groupby('nam').max()
    return data

#%%
# Funtion chia nhỏ các quãng thay đổi của chỉ số.
# Gọi việc này là làm mượt số liệu cho một địa phương
def lam_muot_du_lieu(data, var, by, fps, ascending=True):
    # data: là bộ dữ liệu
    # var: là cột số cần làm mượt. 1 cột hoặc 1 list các cột
    # by: là cột cố định trong khi làm mượt
    # fps: số  lần chia nhỏ dữ liệu (1 lần là 1 khung hình)
    # ==> Hàm này sẽ làm mượt dữ liệu cột bar theo từng bước thay đổi của cột by,
    # ==> mỗi bước thay đổi của cột by sẽ chia nhỏ cột var thành fps lần thay đổi (VD: từ 1-5 chia nhỏ thành: 1-2-3-4-5)
    # ==> data sẽ được bổ sung thêm cột fps - sẽ là cột để làm giá trị chạy cho biểu đồ.
    # ==> cột var được tạo thêm các giá trị trung gian để làm mượt giữ liệu

    # Tạo list rỗng để chứa các giá trị
    list_of_dic = []
    # danh sách các cột không thay đổi
    column_names = list(data.columns.values)
    if type(var) != list:
        var = [var]
    for i in var:
        column_names.remove(i)
    # sắp xếp dữ liệu theo cột by cố định
    data = data.sort_values(by=by, ascending=ascending).reset_index(drop=True)

    # Vòng lặp đầu tiên đi từ đầu đến cuối bộ dữ liệu
    # Lưu ý: dữ liệu cần được sắp
    for i in range(data.shape[0]):
        if i == 0:
            # Tại hàng đầu tiên của dữ liệu
            dic = {}
            for c in column_names:
                dic[c] = data[c][i]
            for v in var:
                dic[v] = data[v][i]
            dic['fps'] = str(data[by][i]) + '-' + str(fps)
            list_of_dic.append(dic)
        else:
            # Tại các dòng tiếp theo
            # Lặp để tạo thêm các số liệu trung gian nhằm tăng độ mượt
            # Với số khung hình (fps) được chọn
            for j in range(fps):
                #
                dic = {}
                for c in column_names:
                    dic[c] = data[c][i]
                for v in var:
                    phan_du = data[v][i] - data[v][i - 1] #
                    value = data[v][i - 1] + (j/(fps-1)) * phan_du * data['toc_do_truot'][i]
                    if phan_du > 0:
                        if value >= data[v][i]:
                            value = data[v][i]
                    if phan_du < 0:
                        if value <= data[v][i]:
                            value = data[v][i]
                    dic[v] = value
                dic['fps'] = str(data[by][i]) + '-' + '0' * (len(str(fps)) - len(str(j + 1))) + str(j + 1)
                list_of_dic.append(dic)
    return pd.DataFrame(list_of_dic)


#%%
def ve_lollipop_plot(data, xmin, xmax):
    xmean = xmin + (xmax - xmin)/2
    plt.style.use('dark_background')
    fig = plt.figure(figsize  = (5, 7), dpi = 150)
    plt.subplot(111)
    mau_line = 'whitesmoke'
    mau_pci = 'peru'
    mau_pci_khoi_dau = 'skyblue'
    size_tieu_de = 10
    size_diem = 8
    size_pci = 4.5
    mau_tieu_de = 'w'
    mau_giam = 'indianred'
    mau_tang = 'seagreen'

    # Điểm PCI năm khởi đầu
    plt.scatter(x = data['pci_khoi_dau'], y = data['xep_hang_bien_chay'],
                color= mau_pci_khoi_dau, alpha=1, s = size_diem, label='Năm 2006')

    # Line nối và pci chạy loại trừ năm 2006
    if data['fps'].iloc[0].split('-')[0] != '2006':
        plt.hlines(y=data['xep_hang_bien_chay'],
                   xmin= data['pci_khoi_dau'], xmax=data['pci_bien_chay'],
                   color= mau_line, alpha=0.6)
        # Điểm PCI năm hiện tại
        plt.scatter(x = data['pci_bien_chay'], y = data['xep_hang_bien_chay'],
                    color= mau_pci, alpha=1, s = size_diem,
                    label='Năm' + str(data['nam'].iloc[0]))


    # Cố định trục
    plt.xlim((xmin, xmax+2))
    plt.ylim((-64, 2))

    # Canh chỉnh tiêu đề
    plt.title('CHỈ SỐ NĂNG LỰC CẠNH TRANH CẤP TỈNH (PCI)',
              fontweight="bold", fontsize= size_tieu_de, color= mau_tieu_de)

    if data['nam'].iloc[0] != 2006:
        plt.text(x = xmean - 2, y = 1, s = str(data['nam_khoi_dau'].iloc[0]),
                 color= mau_pci_khoi_dau, va="center", ha="right", fontweight="bold", fontsize= size_tieu_de)
        plt.scatter(x= xmean -2 -8 , y= 1.1,
                    color=mau_pci_khoi_dau, alpha=1, s=size_diem * 7, label='Năm 2006')
        plt.text(x = xmean , y = 1, s = '-',
                 va="center", ha="center", fontweight="bold", fontsize= size_tieu_de, color= mau_tieu_de)
        plt.text(x = xmean + 2, y = 1, s = str(data['nam'].iloc[0]),
                 color= mau_pci, va="center", ha="left", fontweight="bold", fontsize= size_tieu_de)
        plt.scatter(x= xmean +2 +8 , y= 1.1,
                    color=mau_pci, alpha=1, s=size_diem * 7, label='Năm' + str(data['nam'].iloc[0]))
    else:
        plt.text(x = xmean, y = 1, s = str(data['nam_khoi_dau'].iloc[0]),
                 color= mau_pci_khoi_dau, va="center", ha="center", fontweight="bold", fontsize= size_tieu_de)
        plt.scatter(x= xmean -8 , y= 1.1,
                    color=mau_pci_khoi_dau, alpha=1, s=size_diem * 7, label='Năm 2006')

    # Vẽ tên địa phương và xếp hạng
    # Nếu tăng hạng màu xanh, giảm màu đỏ và giữ nguyên hạng là màu trắng (so với 2016)
    for i in range(data.shape[0]):
        # Điểm PCI

        if data['pci_bien_chay'].iloc[i] >= data['pci_khoi_dau'].iloc[i]:
            toa_do_x_pci_khoi_dau = data['pci_khoi_dau'].iloc[i] - 1
            ha_pci_khoi_dau = 'right'
            toa_do_x_pci_hien_tai = data['pci_bien_chay'].iloc[i] + 1
            ha_pci_hien_tai = 'left'
        else:
            toa_do_x_pci_khoi_dau = data['pci_khoi_dau'].iloc[i] + 1
            ha_pci_khoi_dau = 'left'
            toa_do_x_pci_hien_tai = data['pci_bien_chay'].iloc[i] - 1
            ha_pci_hien_tai = 'right'

        # Line mờ hiệu ứng dễ nhìn
        diem_ket_line_mo_1 = min(toa_do_x_pci_khoi_dau, toa_do_x_pci_hien_tai) - 4.5
        diem_ket_line_mo_1 = diem_ket_line_mo_1 if diem_ket_line_mo_1 > (1.2 * xmin) else (1.2 * xmin)
        plt.hlines(y=data['xep_hang_bien_chay'].iloc[i],
                   xmin= 1.2 * xmin, xmax= diem_ket_line_mo_1,
                   linewidths = 0.3, linestyles = 'dotted', color=mau_line, alpha=0.45)

        # Điểm PCI năm khởi đầu
        plt.text(x= toa_do_x_pci_khoi_dau, y= data['xep_hang_bien_chay'].iloc[i],
                 s= str(data['pci_khoi_dau'].iloc[i]),
                 color= mau_pci_khoi_dau, va="center", ha= ha_pci_khoi_dau, fontsize= size_pci, stretch='ultra-condensed')

        # Điểm PCI năm hiện tại
        if data['fps'].iloc[0].split('-')[0] != '2006':
            plt.text(x= toa_do_x_pci_hien_tai, y= data['xep_hang_bien_chay'].iloc[i],
                     s= str(data['pci'].iloc[i]),
                    color= mau_pci, va="center", ha= ha_pci_hien_tai, fontsize= size_pci, stretch='ultra-condensed')

        # Xếp hạng
        str_xep_hang = str(data['xep_hang'].iloc[i])
        plt.text(x=xmin * 0.82, y=data['xep_hang'].iloc[i] * -1,
                 s=str_xep_hang + '. ',
                 color=mau_tieu_de, va="center", ha="right", fontsize= 5,
                 fontweight="bold", stretch='ultra-condensed', alpha=0.7)

        # Điều kiện để xuất hiện thay đổi thứ bậc số với năm trước
        str_dia_phuong = str(data['dia_phuong'].iloc[i])
        mau_chu = mau_tieu_de
        if data['thay_doi_hang_theo_nam'].iloc[i] < 0:
            mau_chu = mau_tang
        elif data['thay_doi_hang_theo_nam'].iloc[i] > 0:
            mau_chu = mau_giam

        plt.text(x=xmin * 0.85, y=data['xep_hang_bien_chay'].iloc[i],
                s= str_dia_phuong,
                color=mau_chu, va="center", ha="left", fontsize= size_pci,
                fontweight="bold", stretch='ultra-condensed', alpha=0.7)

        xuat_hien_tang_hang = data['pci'].iloc[i] == data['pci_bien_chay'].iloc[i]
        khac_nam_2006 = data['fps'].iloc[0].split('-')[0] != '2006'
        if khac_nam_2006:
            if xuat_hien_tang_hang:
                # Thay đổi màu chữ của tăng giảm thứ hạng so với năm trước
                if data['thay_doi_hang_theo_nam'].iloc[i] < 0:
                    str_thay_doi_hang_theo_nam = '+' + str(data['thay_doi_hang_theo_nam'].iloc[i] * -1)
                elif data['thay_doi_hang_theo_nam'].iloc[i] == 0:
                    str_thay_doi_hang_theo_nam = ''
                else:
                    str_thay_doi_hang_theo_nam = str(data['thay_doi_hang_theo_nam'].iloc[i] * -1)
            else:
                str_thay_doi_hang_theo_nam = ''

            plt.text(x=xmin * 0.72, y=data['xep_hang_bien_chay'].iloc[i],
                    s= str_thay_doi_hang_theo_nam,
                    color=mau_chu, va="center", ha="right", fontsize= size_pci,
                    fontweight="bold", stretch='ultra-condensed', alpha=0.7)

        # Thay đổi màu chữ của tăng giảm thứ hạng so với 2006
        if khac_nam_2006:
            if xuat_hien_tang_hang:
                thay_doi_xep_hang_vs_2006 = data['xep_hang'].iloc[i] - data['xep_hang_khoi_dau'].iloc[i]
                thay_doi_xep_hang_vs_2006 = thay_doi_xep_hang_vs_2006 * -1

                # Thay đổi màu chữ của tăng giảm thứ hạng so với 2006
                if thay_doi_xep_hang_vs_2006 < 0:
                    str_thay_doi_xep_hang = str(thay_doi_xep_hang_vs_2006)
                    mau_thay_doi_xep_hang = mau_giam
                elif thay_doi_xep_hang_vs_2006 == 0:
                    str_thay_doi_xep_hang = '-'
                    mau_thay_doi_xep_hang = mau_tieu_de
                else:
                    str_thay_doi_xep_hang = '+' + str(thay_doi_xep_hang_vs_2006)
                    mau_thay_doi_xep_hang = mau_tang
            else:
                str_thay_doi_xep_hang = '-'
                mau_thay_doi_xep_hang = mau_tieu_de

            plt.text(x=xmax + 2, y=data['xep_hang'].iloc[i] * -1,
                     s=str_thay_doi_xep_hang,
                     color= mau_thay_doi_xep_hang, va="center", ha="right", fontsize= size_pci,
                     fontweight="bold", stretch='ultra-condensed', alpha=0.7)


        if khac_nam_2006:
            diem_ket_line_mo_2 = max(toa_do_x_pci_khoi_dau, toa_do_x_pci_hien_tai) + 4.5
            diem_ket_line_mo_2 = diem_ket_line_mo_2 if diem_ket_line_mo_2 < (xmax + 2) else (xmax + 2)
            plt.hlines(y=data['xep_hang_bien_chay'].iloc[i],
                        xmin= (xmax), xmax=diem_ket_line_mo_2,
                        linewidths=0.3, linestyles='dotted', color=mau_line, alpha=0.45)

            # Ghi chú thay đổi thứ hạng
            str_ghi_chu_thu_hang = 'THỨ HẠNG ' + str(data['nam'].iloc[0])  + '\nSO VỚI 2006'
            plt.text(x=xmax + 3, y= 0.5,
                     s= str_ghi_chu_thu_hang,
                     color= mau_tieu_de, va="center", ha="right", fontsize= size_pci,
                     stretch='ultra-condensed', alpha=0.5)

    # Tạo chữ ký
    plt.text(xmax + 2, -65,
             'Thể hiện: Võ Văn Thương\nNguồn dữ liệu: http://pcivietnam.org',
             color= mau_tieu_de, va="center", ha="right", fontsize = 5)

    plt.axis('off')
    fig.canvas.draw()
    image_from_plot = fig.canvas.tostring_rgb()
    image_from_plot = np.frombuffer(image_from_plot, dtype=np.uint8)
    w, h = fig.canvas.get_width_height()
    image_from_plot = image_from_plot.reshape(h, w, 3)
    # Conver qua mảng BGR theo định dạng của OpenCV
    image_from_plot = cv2.cvtColor(image_from_plot, cv2.COLOR_RGB2BGR)
    plt.close(fig)
    return image_from_plot

In [3]:
#%%
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import cv2

#%%
# Các thông tin của bảng xếp hạng chỉ số năng lực cạnh tranh cấp tỉnh công bố tại website: http://pcivietnam.org
url = 'http://pcivietnam.org/bang-xep-hang/?khu_vuc={}&chi_so={}&nam={}'
nam = range(2006,2018+1)
khu_vuc = 0
chi_so = 'pci'

#%%
# Lấy dữ liệu về
pci_vn = lay_pci_vn(url, nam, khu_vuc, chi_so)

#%%
# Lưu lại để sử dụng lần sau
# pci_vn.to_csv('data/pci_vn.csv')
# pci_vn = pd.read_csv('data/pci_vn.csv', index_col= 0)

#%%
# Cột Nhóm xếp hạng không đầy đủ -> không sử dụng được -> xoá
pci_vn = pci_vn.drop(columns = 'Nhóm xếp hạng')
# Đổi tên biến
pci_vn = pci_vn.rename(columns = {'Địa phương': 'dia_phuong', 'Điểm số PCI': 'pci', 'Xếp hạng': 'xep_hang'})
# Làm tròn chỉ số PCI về 2 chữ số thập thân
pci_vn['pci'] = pci_vn['pci'].apply(lambda x: round(x, 2))

#%%
# Kiểm tra danh sách địa phương thì danh sách địa phương có đến 65 giá trị
# => Có 2 địa phương bị ghi sai lỗi chính tả (hoặc cách ghi không đồng nhất): Da Nang và Ha Nam
# Chuẩn hoá tên hai địa phương viết không đồng nhất
pci_vn.loc[pci_vn['dia_phuong'] == 'Da Nang', 'dia_phuong'] = 'Đà Nẵng'
pci_vn.loc[pci_vn['dia_phuong'] == 'Ha Nam', 'dia_phuong'] = 'Hà Nam'

#%%
# Xác định xong bảng giá trị khởi đầu.
pci_vn_dau = xac_dinh_khoi_dau(pci_vn)
pci_vn_dau.rename(columns = {'pci': 'pci_khoi_dau', 'nam': 'nam_khoi_dau', 'xep_hang': 'xep_hang_khoi_dau'}, inplace= True)
# Lúc này nhận thấy bảng giá trị khởi đầu chính là năm 2006.
# Chỉ số PCI đã thu thập từ năm 2005 trên 42 tỉnh thành tuy nhiên giá trị năm 2015 lại không được đưa vào bảng xếp hạng.
# Tuy vậy funtion này cũng củng cố thêm niềm tin bảng giá trị này chính là bảng giá trị khởi đầu.

#%%
# Tạo cột số thứ hạng thay đổi theo năm và điểm thay đổi theo từng năm
pci_vn = bo_sung_thay_doi_theo_nam(pci_vn)

#%%
thay_doi_toi_da = tim_thay_doi_hang_toi_da_theo_nam(pci_vn)

#%%
# Xác định tốc độ trượt trên trục thứ hạng (trục y)
# bằng cách nhân với giá trị tuyệt đối của thương: số hạng thay đổi tối đa / số hạng thay đổi của nước hiện tại

# Thêm cột thay đổi tối đa mỗi năm
pci_vn = pci_vn.join(other= thay_doi_toi_da, on= 'nam')

# Cột tốc độ trượt
pci_vn['toc_do_truot'] = pci_vn['thay_doi_toi_da'] / pci_vn['thay_doi_hang_theo_nam'].abs()

#%%
# Điều chỉnh giá trị của tốc độ đối với năm 2006 bằng 0
pci_vn['toc_do_truot'].loc[(pci_vn['thay_doi_hang_theo_nam'] == 0) & (pci_vn['nam'] == 2006)] = 0

#%%
# Gán các giá trị dương vô cùng bằng nan
pci_vn['toc_do_truot'].loc[(pci_vn['toc_do_truot'] == np.inf)] = np.nan

#%%
# Ở mỗi năm còn lại các giá trị không thay đổi hạng được gán tốc độ bằng tốc độ tối đa
for i in pci_vn.nam.unique():
    pci_vn['toc_do_truot'].loc[(pci_vn['thay_doi_hang_theo_nam'] == 0) & (pci_vn['nam'] == i)] = pci_vn['toc_do_truot'].loc[pci_vn['nam'] == i].max()

#%%
# Tạo biến PCI và biến xếp hạng riêng để làm giá trị chạy
# Biến PCI và biến xếp hạng gốc giữ lại nhằm lấy giá trị hiển thị
pci_vn['pci_bien_chay'] = pci_vn['pci']
pci_vn['xep_hang_bien_chay'] = pci_vn['xep_hang']

#%%
# Làm mượt dữ liệu
pci_vn_da_muot = pd.DataFrame()
for i in pci_vn['dia_phuong'].unique():
    data = pci_vn[pci_vn['dia_phuong'] == i]
    data = lam_muot_du_lieu(data=data, var=['pci_bien_chay', 'xep_hang_bien_chay'], by='nam', fps=80)
    pci_vn_da_muot = pci_vn_da_muot.append(data)

#%%
# Thêm các cột thông tin lúc khởi đầu thu thập vào dataframe
pci_vn_da_muot = pci_vn_da_muot.merge(pci_vn_dau, on= 'dia_phuong')

#%%
# Chuyển cột xếp hạng thành giá trị âm để đẩy cột hạng cao nhất lên trên
pci_vn_da_muot['xep_hang_bien_chay'] = pci_vn_da_muot['xep_hang_bien_chay'] * -1

#%%
# Tạo các giá trị trục cho biểu đồ
xmin = pci_vn_da_muot[['pci_khoi_dau', 'pci_bien_chay']].min().min()
xmin = xmin * 0.75
xmax = pci_vn_da_muot[['pci_khoi_dau', 'pci_bien_chay']].max().max()
xmax = xmax * 1.1




/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [4]:
#%%
fps = pci_vn_da_muot['fps'].unique()
fps = np.sort(fps)

list_plot_array = []
i = 0
for f in fps:
    data = pci_vn_da_muot[pci_vn_da_muot['fps'] == f]
    plot_data = ve_lollipop_plot(data, xmin, xmax)
    list_plot_array.append(plot_data)
    i += 1
    print('Hình thứ:', i, '/', len(fps))
    if f.split('-')[1] == '80':
        if f.split('-')[1] == '2006':
            so_vong = 120
        else:
            so_vong = 60
        for j in range(so_vong):
            list_plot_array.append(plot_data)

# %% RENDER VIDEO
height, width, layers = list_plot_array[0].shape
size = (width, height)
video_name = '/content/gdrive/My Drive/Colab Notebooks/PCI_VN/Lollipop_PCI_VN-2006-2018.avi'
video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
for i in range(len(list_plot_array)):
    video.write(list_plot_array[i])
video.release()

Hình thứ: 1 / 961
Hình thứ: 2 / 961
Hình thứ: 3 / 961
Hình thứ: 4 / 961
Hình thứ: 5 / 961
Hình thứ: 6 / 961
Hình thứ: 7 / 961
Hình thứ: 8 / 961
Hình thứ: 9 / 961
Hình thứ: 10 / 961
Hình thứ: 11 / 961
Hình thứ: 12 / 961
Hình thứ: 13 / 961
Hình thứ: 14 / 961
Hình thứ: 15 / 961
Hình thứ: 16 / 961
Hình thứ: 17 / 961
Hình thứ: 18 / 961
Hình thứ: 19 / 961
Hình thứ: 20 / 961
Hình thứ: 21 / 961
Hình thứ: 22 / 961
Hình thứ: 23 / 961
Hình thứ: 24 / 961
Hình thứ: 25 / 961
Hình thứ: 26 / 961
Hình thứ: 27 / 961
Hình thứ: 28 / 961
Hình thứ: 29 / 961
Hình thứ: 30 / 961
Hình thứ: 31 / 961
Hình thứ: 32 / 961
Hình thứ: 33 / 961
Hình thứ: 34 / 961
Hình thứ: 35 / 961
Hình thứ: 36 / 961
Hình thứ: 37 / 961
Hình thứ: 38 / 961
Hình thứ: 39 / 961
Hình thứ: 40 / 961
Hình thứ: 41 / 961
Hình thứ: 42 / 961
Hình thứ: 43 / 961
Hình thứ: 44 / 961
Hình thứ: 45 / 961
Hình thứ: 46 / 961
Hình thứ: 47 / 961
Hình thứ: 48 / 961
Hình thứ: 49 / 961
Hình thứ: 50 / 961
Hình thứ: 51 / 961
Hình thứ: 52 / 961
Hình thứ: 53 / 961
Hì